In [1]:
library(SpatialPCA)
library(slingshot)

Loading required package: princurve

Loading required package: TrajectoryUtils

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLog

In [2]:
exp_df <- read.csv("simulated_expression.csv", row.names = 1)

In [3]:
rownames(exp_df) <- gsub("_", "-", rownames(exp_df))

In [4]:
head(exp_df)

,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,cell9,cell10,⋯,cell445,cell446,cell447,cell448,cell449,cell450,cell451,cell452,cell453,cell454
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
A1-TF1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2-TF1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2-TF2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2-TF3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2-TF4,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A3-TF1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [ ]:
meta_info_df <- read.csv("simulation_disconnected_ontrac_input.csv", row.names = 1)

In [7]:
xy_coods <- as.matrix(meta_info_df[colnames(exp_df),c('x', 'y')])

In [8]:
head(xy_coods)

,x,y
cell1,-0.400000000,0.93397740
cell2,0.400000000,-0.97018565
cell3,-0.405309735,-0.82975244
cell4,0.000000000,-0.02347488
cell5,0.405357143,-0.98190729
cell6,0.001769912,0.82813316


In [9]:
spatialPCA_obj <- CreateSpatialPCAObject(counts=as.matrix(exp_df), location=xy_coods, project = "SpatialPCA",gene.type="spatial",sparkversion="spark",numCores_spark=5,gene.number=3000, customGenelist=NULL,min.loctions = 2, min.features=2)


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


## Use SCTransform function in Seurat to normalize data. 


Running SCTransform on assay: RNA

Running SCTransform on layer: counts

vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

`vst.flavor` is set to 'v2' but could not find glmGamPoi installed.
Please install the glmGamPoi package for much faster estimation.
--------------------------------------------
install.packages('BiocManager')
BiocManager::install('glmGamPoi')
--------------------------------------------
Falling back to native (slower) implementation.


Variance stabilizing transformation of count matrix of size 89 by 454

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 89 genes, 454 cells

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs produced”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs pr

## Use spark.test function in SPARK package to select spatially variable genes. 
## ===== SPARK INPUT INFORMATION ==== 
## number of total samples:  454 
## number of total features:  87 
## number of adjusted covariates:  0 
# fitting normalized count-based spatial model under the null hypothesis ... 


Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’



## testing Gaussian kernel: 1...
## testing Periodic kernel: 1...
## testing Gaussian kernel: 2...
## testing Periodic kernel: 2...
## testing Gaussian kernel: 3...
## testing Periodic kernel: 3...
## testing Gaussian kernel: 4...
## testing Periodic kernel: 4...
## testing Gaussian kernel: 5...
## testing Periodic kernel: 5...
## Identified  79  spatial genes through spark.test function. 
The  number of significant spatial genes is less than the specified number of spatial genes. 
## Using  79  significant spatially variable genes. 


In [10]:
spatialPCA_obj <- SpatialPCA_buildKernel(spatialPCA_obj, kerneltype="gaussian", bandwidthtype="SJ",bandwidth.set.by.user=NULL)
spatialPCA_obj <- SpatialPCA_EstimateLoading(spatialPCA_obj,fast=FALSE,SpatialPCnum=10) 
spatialPCA_obj <- SpatialPCA_SpatialPCs(spatialPCA_obj, fast=FALSE)

## Selected kernel type is:  gaussian  
## The bandwidth is:  0.0467434969865739  
## Calculating kernel matrix
## Finished calculating kernel matrix.
[1] "Eigen decomposition on kernel matrix!"
[1] "Using all eigenvectors and eigenvalues in the Kernel matrix!"


In [11]:
clusterlabel= walktrap_clustering(clusternum=6,
                                  latent_dat=spatialPCA_obj@SpatialPCs,
                                  knearest=round(sqrt(dim(spatialPCA_obj@SpatialPCs)[2]))
                                 ) 

In [12]:
sce <- SingleCellExperiment(assays = as.matrix(exp_df))

In [13]:
reducedDims(sce) <- SimpleList(DRM = t(spatialPCA_obj@SpatialPCs))

In [14]:
colData(sce)$clusterlabel <- factor(clusterlabel)    
sce  <-slingshot(sce, clusterLabels = 'clusterlabel', reducedDim = 'DRM',start.clus="3" ) 
# in this data we set white matter region as start cluster, one can change to their preferred start region 

# summary(sce@colData@listData)
pseudotime_traj1 <- sce@colData@listData$slingPseudotime_1 # in this data only one trajectory was inferred


In [15]:
write.csv(pseudotime_traj1, "pseudotime_spatial_pca.csv")